In [1]:
# Tried to auto-check PEP8 but program crashes with a bug
# %load_ext pycodestyle_magic
# %pycodestyle_on
# %pycodestyle_off

In [2]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Names of all csv files
all_files = [
    'lifeExpectancyAtBirth', 'HALElifeExpectancyAtBirth',
    'maternalMortalityRatio', 'birthAttendedBySkilledPersonal',
    'infantMortalityRate', 'neonatalMortalityRate',
    'under5MortalityRate', 'incedenceOfMalaria',
    'incedenceOfTuberculosis', 'hepatitusBsurfaceAntigen',
    'interventionAgianstNTDs', 'newHivInfections',
    '30-70cancerChdEtc', 'crudeSuicideRates',
    'AlcoholSubstanceAbuse', 'roadTrafficDeaths',
    'reproductiveAgeWomen', 'adolescentBirthRate',
    'uhcCoverage', 'dataAvailibilityForUhc',
    'population10SDG3.8.2', 'population25SDG3.8.2',
    'airPollutionDeathRate', 'mortalityRateUnsafeWash',
    'mortalityRatePoisoning', 'tobaccoAge15',
    'medicalDoctors', 'nursingAndMidwife',
    'Dentists', 'Pharmacists',
    'eliminateViolenceAgainstWomen', 'basicDrinkingWaterServices',
    'atLeastBasicSanitizationServices', 'safelySanitization',
    'basicHandWashing', 'cleanFuelAndTech',
]

In [4]:
class DataAnalysis():
    """Utility class for data analysis tools."""
    def __init__(self):
        # RE expression for bracketed information within the dataset.
        self.rebrackets = re.compile(r'\[[\d.]+\-[\d.]+\]')

    def get_data(self, fname):
        # Utility function that takes file name and returns a dataframe.
        dfnew = pd.read_csv(os.path.join('dataset', f"{fname}.csv"))
        return dfnew

    def remove_brackets(self, numstr):
        # Utility function to remove brackets from a value
        # if the value is string.
        try:
            re_match = self.rebrackets.search(numstr)
            try:
                numstr = numstr[:re_match.start()].strip()
                return float(numstr)
            except AttributeError:
                return np.nan
        except TypeError:
            return numstr

    def column_info(self, df):
        # Utility function to print column general info.
        print(f"TOTAL - {len(df)}")
        for col in df.columns:
            colset = set(df[col])
            colset_len = len(colset)
            print(f"{col}: {colset_len}")
            if pd.api.types.is_numeric_dtype(df[col]):
                print(' - '.join((
                    '',
                    str(df[col].min()),
                    str(df[col].median()),
                    str(df[col].max())
                    )))
            if colset_len < 10:
                for colset_ele in colset:
                    print(f" - {colset_ele}: {sum(df[col] == colset_ele)}")

    def index_change(self, df, cols=['Period'], values=[None]):
        # Index change while filtering into one entry per index.
        for col, value in zip(cols, values):
            if value == None:
                df = df[df[col] == df[col].max()]
            else:
                df = df[df[col] == value]
        return df.set_index('Location')
    
    def simplify_df(self, df):
        # Utility function to simplify dataframe and
        # only retain one data column.
        col_str = df['Indicator'][0]
        df = df[['First Tooltip']]
        df.columns = [col_str]
        return df
    

In [5]:
da = DataAnalysis()

In [6]:
df = da.get_data('cleanFuelAndTech')
da.column_info(df)
df.head()

TOTAL - 3610
Location: 191
Indicator: 1
 - Proportion of population with primary reliance on clean fuels and technologies (%): 3610
Period: 19
 - 2000 - 2009.0 - 2018
First Tooltip: 91
 - 5 - 81.0 - 95


,Location,Indicator,Period,First Tooltip
0,Afghanistan,Proportion of population with primary reliance...,2018,37
1,Afghanistan,Proportion of population with primary reliance...,2017,34
2,Afghanistan,Proportion of population with primary reliance...,2016,31
3,Afghanistan,Proportion of population with primary reliance...,2015,29
4,Afghanistan,Proportion of population with primary reliance...,2014,27


In [7]:
# Choosing one set of data per location
print('lifeExpectancyAtBirth')
df_life = da.index_change(da.get_data(
    'lifeExpectancyAtBirth'), ['Period', 'Dim1'],
    [None, 'Both sexes'])
da.column_info(df_life)
print('\ncleanFuelAndTech')
df_clean = da.index_change(da.get_data('cleanFuelAndTech'))
da.column_info(df_clean)
print('\nbasicDrinkingWaterServices')
df_drinking = da.index_change(da.get_data(
    'basicDrinkingWaterServices'))
da.column_info(df_drinking)
print('\natLeastBasicSanitizationServices')
df_sanitation = da.index_change(da.get_data(
    'atLeastBasicSanitizationServices'), ['Period', 'Dim1'],
    [None, 'Total'])
da.column_info(df_sanitation)

lifeExpectancyAtBirth
TOTAL - 183
Period: 1
 - 2019 - 2019.0 - 2019
 - 2019: 183
Indicator: 1
 - Life expectancy at birth (years): 183
Dim1: 1
 - Both sexes: 183
First Tooltip: 175
 - 50.75 - 73.74 - 84.26

cleanFuelAndTech
TOTAL - 190
Indicator: 1
 - Proportion of population with primary reliance on clean fuels and technologies (%): 190
Period: 1
 - 2018 - 2018.0 - 2018
 - 2018: 190
First Tooltip: 54
 - 5 - 84.0 - 95

basicDrinkingWaterServices
TOTAL - 190
Period: 1
 - 2017 - 2017.0 - 2017
 - 2017: 190
Indicator: 1
 - Population using at least basic drinking-water services (%): 190
First Tooltip: 144
 - 22.83 - 92.265 - 100.0

atLeastBasicSanitizationServices
TOTAL - 188
Indicator: 1
 - Population using at least basic sanitation services (%): 188
Period: 1
 - 2017 - 2017.0 - 2017
 - 2017: 188
Dim1: 1
 - Total: 188
First Tooltip: 167
 - 7.32 - 89.065 - 100.0


In [8]:
df_life = da.simplify_df(df_life)
df_clean = da.simplify_df(df_clean)
df_drinking = da.simplify_df(df_drinking)
df_sanitation = da.simplify_df(df_sanitation)

In [9]:
#< ---- 20 char ---->< ---- 20 char --->< ---- 20 char ---><9 char->
df = pd.concat([df_life, df_clean, 
                df_drinking, df_sanitation], axis=1)

In [10]:
df[df.isnull().sum(axis=1) > 0]

,Life expectancy at birth (years),Proportion of population with primary reliance on clean fuels and technologies (%),Population using at least basic drinking-water services (%),Population using at least basic sanitation services (%)
Argentina,76.58,95.0,99.64,NaN
Brunei Darussalam,74.32,95.0,99.90,NaN
Bulgaria,75.07,NaN,98.05,86.00
Central African Republic,53.10,5.0,NaN,NaN
Cuba,77.76,NaN,90.31,92.81
Eritrea,64.08,9.0,NaN,NaN
Lebanon,76.44,NaN,92.60,98.48
Libya,75.78,NaN,98.53,100.00
Andorra,NaN,95.0,100.00,100.00
Cook Islands,NaN,77.0,99.88,97.63


In [11]:
# Remove rows with NAN values.
incomplete_rows = df[df.isnull().sum(axis=1) > 0].index
df = df.drop(incomplete_rows)
# Store full column names just in case.
col_full = df.columns
df.columns = ['Life expectancy', 'Clean technologies',
             'Drinking-water', 'Sanitation']

In [12]:
df

,Life expectancy,Clean technologies,Drinking-water,Sanitation
Afghanistan,63.21,37.0,57.32,43.42
Albania,78.00,80.0,89.88,97.72
Algeria,77.13,95.0,88.69,87.59
Angola,63.06,48.0,27.44,49.88
Antigua and Barbuda,76.45,95.0,96.74,87.50
...,...,...,...,...
Venezuela (Bolivarian Republic of),73.95,95.0,95.72,93.94
Viet Nam,73.74,64.0,92.64,83.52
Yemen,66.63,60.0,54.70,59.05
Zambia,62.45,13.0,41.95,26.37


In [13]:
# Linear regression on the four columns.
from sklearn.linear_model import LinearRegression

In [14]:
X = df.drop('Life expectancy', axis=1)
y = df[['Life expectancy']]
reg = LinearRegression().fit(X, y)
reg.score(X, y)

0.7224114226667779

In [15]:
reg.coef_

array([[0.0615704 , 0.11546261, 0.05591629]])

In [16]:
reg.intercept_

array([55.12865485])

In [17]:
df.loc['Zimbabwe']

Life expectancy       60.68
Clean technologies    29.00
Drinking-water        49.80
Sanitation            36.22
Name: Zimbabwe, dtype: float64

In [18]:
reg.predict([df.drop('Life expectancy', axis=1).loc['Zimbabwe']])

array([[64.6895228]])

In [23]:
for filename in all_files:
    print(filename)
    da.column_info(da.get_data(filename))

lifeExpectancyAtBirth
TOTAL - 2197
Location: 184
Period: 5
 - 1920 - 2010.0 - 2019
 - 1920: 1
 - 2019: 549
 - 2000: 549
 - 2010: 549
 - 2015: 549
Indicator: 1
 - Life expectancy at birth (years): 2197
Dim1: 3
 - Both sexes: 733
 - Male: 732
 - Female: 732
First Tooltip: 1580
 - 27.97 - 71.83 - 86.94
HALElifeExpectancyAtBirth
TOTAL - 2196
Location: 184
Period: 4
 - 2000 - 2012.5 - 2019
 - 2000: 549
 - 2010: 549
 - 2019: 549
 - 2015: 549
Indicator: 1
 - Healthy life expectancy (HALE) at birth (years): 2196
Dim1: 3
 - Both sexes: 732
 - Male: 732
 - Female: 732
First Tooltip: 1492
 - 25.37 - 63.11 - 75.48
maternalMortalityRatio
TOTAL - 3294
Location: 184
Period: 18
 - 2000 - 2008.5 - 2017
Indicator: 1
 - Maternal mortality ratio (per 100 000 live births): 3294
First Tooltip: 2421
birthAttendedBySkilledPersonal
TOTAL - 1755
Location: 186
Period: 20
 - 2000 - 2009.0 - 2019
Indicator: 1
 - Births attended by skilled health personnel (%): 1755
First Tooltip: 348
 - 5.7 - 99.0 - 100.0
infantMo